In [1]:
import pandas as pd
import numpy as np
import multiprocessing
from collections import defaultdict
from time import sleep
import lyricsgenius
import re
import urllib.request
from bs4 import BeautifulSoup
from collections import Counter

In [2]:
import pickle
art_song_lyrics = pickle.loads(open("../data/pickle/GeniusGithub_found.pickle", "rb").read())
missing_art_song_lyrics = pickle.loads(open("../data/pickle/GeniusGithub_missing.pickle", "rb").read())

In [3]:
BB = pd.read_csv("../data/Hot Stuff.csv")
BB_artist_song_pair = list(BB.groupby(["Performer", "Song"]).count().index)
print("Ttoal Artist-Song Pair:  {}".format(len(BB_artist_song_pair)))

Ttoal Artist-Song Pair:  27949


In [4]:
# art_song_lyrics = defaultdict(dict)
# missing_art_song_lyrics = defaultdict(set)

## Use Genius DataBase

In [5]:
genius = lyricsgenius.Genius("dxvLAG0Jhx6B01DK9pLzusCIFlboFV996sOvwEAPQbON-MUPz2vLgu9_z7u-hODo", verbose=False)
def Genius_scrape(ArtSongPair):
    a, s = ArtSongPair
    try:
        song = genius.search_song(s, a)
        if song == None:
            return [a, s, None]
        else:
            return [song, a, s]
    except:
        print("Oof chill boy. Too many requests")
        sleep(np.random.random() * 10)
        return Genius_scrape(ArtSongPair)

In [6]:
# with multiprocessing.Pool(processes=multiprocessing.cpu_count()*16) as pool:
#     try:
#         count = 0; found = 0; missing = 0
#         for val1, val2, val3 in pool.imap_unordered(Genius_scrape, BB_artist_song_pair):
#             if type(val1) != str:
#                 art_song_lyrics[val2][val3] = {"lyrics": val1.lyrics, "url": val1.url, "obj": val1}
#                 found += 1
#             else:
#                 missing_art_song_lyrics[val1].add(val2)
#                 missing += 1
#             count += 1
#             if count % 1000 == 0:
#                 print("Till {}:    Found: {}  Missing: {}".format(count, found, missing))
#     except AssertionError as e:
#         print(e)
#     pool.terminate()

In [28]:
genious_found_art_song_pair = [[i, j] for i in art_song_lyrics.keys() for j in art_song_lyrics[i].keys()]
missing_art_song_pair = [[i, j] for i in missing_art_song_lyrics.keys() for j in missing_art_song_lyrics[i]]
print("Total:  {}    No.Found:  {},  No.Missing:  {}".format(len(BB_artist_song_pair), len(genious_found_art_song_pair), len(missing_art_song_pair)))
# assert(len(BB_artist_song_pair) == len(genious_found_art_song_pair) + len(missing_art_song_pair))

Total:  27949    No.Found:  25858,  No.Missing:  2106


In [8]:
# import pickle
# pickle.dump(art_song_lyrics, open("../data/pickle/GeniusGithub_found.pickle", "wb"))
# pickle.dump(missing_art_song_lyrics, open("../data/pickle/GeniusGithub_missing.pickle", "wb"))

## Check Validity (Artist similarity > 0.85 & SongName similarity > 0.85)

In [78]:
from Levenshtein import ratio
genius_verify_art_song_found = defaultdict(dict)
genius_verify_art_song_missing = defaultdict(dict)
for i in art_song_lyrics.keys():
    for j in art_song_lyrics[i].keys():
        if ratio(art_song_lyrics[i][j]["obj"].artist.lower(), i.lower()) > 0.85 and ratio(art_song_lyrics[i][j]["obj"].title.lower(), j.lower()) > 0.85:
            genius_verify_art_song_found[i][j] = dict(art_song_lyrics[i][j])
        else:
            genius_verify_art_song_missing[i][j] = dict(art_song_lyrics[i][j])
            

In [80]:
verified_song_pairs = [(i, j) for i in genius_verify_art_song_found.keys() for j in genius_verify_art_song_found[i].keys()]
not_verified_song_pairs = [(i, j) for i in genius_verify_art_song_missing.keys() for j in genius_verify_art_song_missing[i].keys()]
print("Number of Verified Songs:  ", len(verified_song_pairs))
print("Number of Not Verified Songs:  ", len(not_verified_song_pairs))

Number of Verified Songs:   18883
Number of Not Verified Songs:   6975


In [81]:
missing_artist_score = sorted([ratio(i, genius_verify_art_song_missing[i][j]["obj"].artist.lower()) for i in genius_verify_art_song_missing.keys() for j in genius_verify_art_song_missing[i].keys()])

In [82]:
Counter(missing_artist_score).most_common(20)

[(0.25, 224),
 (0.2857142857142857, 202),
 (0.3333333333333333, 186),
 (0.2222222222222222, 163),
 (0.4, 139),
 (0.3076923076923077, 129),
 (0.2, 124),
 (0.2608695652173913, 123),
 (0.0, 120),
 (0.18181818181818182, 112),
 (0.26666666666666666, 103),
 (0.24, 101),
 (0.5, 101),
 (0.23076923076923078, 95),
 (0.2727272727272727, 83),
 (0.3, 80),
 (0.19047619047619047, 79),
 (0.2962962962962963, 78),
 (0.6666666666666666, 78),
 (0.17391304347826086, 76)]

In [83]:
Counter([j for i in genius_verify_art_song_missing.keys() for j in i.split(" ")]).most_common(20)

[('Featuring', 1724),
 ('The', 838),
 ('&', 736),
 ('Lil', 159),
 ('And', 138),
 ('Wayne', 92),
 ('Brown', 82),
 ('Chris', 76),
 ('Drake', 73),
 ('Nicki', 63),
 ('Big', 63),
 ('Young', 61),
 ('Minaj', 59),
 ('Kanye', 57),
 ('West', 57),
 ('With', 56),
 ('Band', 55),
 ('David', 53),
 ('John', 52),
 ('Paul', 51)]

## Rescrpae the not valid one by removing Featuring, with, &

In [14]:
def reformat_aritst_song(artist, song):
    if "Featuring" in artist:
        artist = artist.split("Featuring")[0].strip()
    if "Feat." in artist:
        artist = artist.split("Feat.")[0].strip()
    if "featuring" in artist:
        artist = artist.split("featuring")[0].strip()
    if "&" in artist:
        artist = artist.split("&")[0].strip()
    if "With" in artist:
        artist = artist.split("With")[0].strip()
    if "with" in artist:
        artist = artist.split("with")[0].strip()
    if "(" in artist:
        artist = artist.split("(")[0].strip()
    return [artist, song]

In [15]:
def Genius_scrape_reformat(ArtSongPair):
    a, s = ArtSongPair
    a_reformat, s_reformat = reformat_aritst_song(a, s)
#     print(a_reformat, s_reformat)
    try:
        song = genius.search_song(s_reformat, a_reformat)
        if song == None:
            return [a, s, None]
        else:
            return [song, a, s]
    except Exception as e:
        print(e)
        print("Oof chill boy. Too many requests")
        sleep(np.random.random() * 10)
        return Genius_scrape(ArtSongPair)

In [16]:
with multiprocessing.Pool(processes=multiprocessing.cpu_count()*4) as pool:
    try:
        count = 0; found = 0; missing = 0
        for val1, val2, val3 in pool.imap_unordered(Genius_scrape_reformat, not_verified_song_pairs):
            if type(val1) != str:
                art_song_lyrics[val2][val3] = {"lyrics": val1.lyrics, "url": val1.url, "obj": val1}
                found += 1
            else:
                missing_art_song_lyrics[val1].add(val2)
                missing += 1
            count += 1
            if count % 1000 == 0:
                print("Till {}:    Found: {}  Missing: {}".format(count, found, missing))
    except AssertionError as e:
        print(e)
    pool.terminate()

Till 1000:    Found: 996  Missing: 4
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))
Oof chill boy. Too many requests
Till 2000:    Found: 1995  Missing: 5
Till 3000:    Found: 2993  Missing: 7
Till 4000:    Found: 3991  Missing: 9
Timeout raised and caught:
HTTPSConnectionPool(host='api.genius.com', port=443): Read timed out. (read timeout=5)
'NoneType' object is not subscriptable
Oof chill boy. Too many requests
Till 5000:    Found: 4990  Missing: 10
Timeout raised and caught:
HTTPSConnectionPool(host='api.genius.com', port=443): Read timed out. (read timeout=5)
'NoneType' object is not subscriptable
Oof chill boy. Too many requests
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))
Oof chill boy. Too many requests
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))
Oof chill boy. Too many requests
Till 6000:    Found: 5988  Missing: 12
('Connection abort

In [84]:
genius_verify_art_song_found_reformat = defaultdict(dict)
genius_verify_art_song_missing_reformat = defaultdict(dict)
for i in art_song_lyrics.keys():
    for j in art_song_lyrics[i].keys():
        i_reformat, j_reformat = reformat_aritst_song(i, j)
        if ratio(art_song_lyrics[i][j]["obj"].artist.lower(), i_reformat.lower()) >= 0.80 and ratio(art_song_lyrics[i][j]["obj"].title.lower(), j_reformat.lower()) >= 0.80:
            genius_verify_art_song_found_reformat[i][j] = dict(art_song_lyrics[i][j])
        else:
            genius_verify_art_song_missing_reformat[i][j] = dict(art_song_lyrics[i][j])

In [86]:
verified_song_pairs_reformat = [(i, j) for i in genius_verify_art_song_found_reformat.keys() for j in genius_verify_art_song_found_reformat[i].keys()]
not_verified_song_pairs_reformat = [(i, j) for i in genius_verify_art_song_missing_reformat.keys() for j in genius_verify_art_song_missing_reformat[i].keys()]
print("Number of Verified Songs:  ", len(verified_song_pairs_reformat))
print("Number of Not Verified Songs:  ", len(not_verified_song_pairs_reformat))

Number of Verified Songs:   20191
Number of Not Verified Songs:   5667


In [56]:
len(verified_song_pairs_reformat) + len(not_verified_song_pairs_reformat)

25858

In [57]:
missing_artist_score_reformat = sorted([ratio(i, genius_verify_art_song_missing_reformat[i][j]["obj"].artist.lower()) for i in genius_verify_art_song_missing_reformat.keys() for j in genius_verify_art_song_missing_reformat[i].keys()])

In [58]:
Counter([i for i in missing_artist_score_reformat if i > 0.8]).most_common(20)

[(0.8333333333333334, 151),
 (0.8571428571428571, 96),
 (0.8461538461538461, 87),
 (0.8181818181818182, 50),
 (0.8666666666666667, 38),
 (0.8823529411764706, 35),
 (0.8888888888888888, 29),
 (0.875, 28),
 (0.8235294117647058, 26),
 (0.8076923076923077, 22),
 (0.8064516129032258, 21),
 (0.8095238095238095, 21),
 (0.8125, 21),
 (0.85, 21),
 (0.8421052631578947, 20),
 (0.8620689655172413, 19),
 (0.84, 14),
 (0.8636363636363636, 13),
 (0.8275862068965517, 12),
 (0.8387096774193549, 10)]

In [60]:
Counter([j for i in genius_verify_art_song_missing_reformat.keys() for j in i.split(" ")]).most_common(20)

[('The', 750),
 ('&', 519),
 ('Featuring', 178),
 ('And', 124),
 ('Band', 50),
 ('Johnny', 38),
 ('John', 36),
 ('Of', 34),
 ('Bobby', 34),
 ('His', 32),
 ('Lil', 32),
 ('David', 31),
 ('Paul', 30),
 ('Joe', 29),
 ('James', 28),
 ('and', 28),
 ('With', 27),
 ('Brown', 27),
 ('Billy', 27),
 ('Orchestra', 26)]

In [90]:
pickle.dump(genius_verify_art_song_found_reformat, open("../data/pickle/GeniusGithub_reformat2.pickle", "wb"))

In [43]:
sorted([(ratio(j.lower(), genius_verify_art_song_found_reformat[i][j]["obj"].title.lower()), i.lower(), genius_verify_art_song_found_reformat[i][j]["obj"].artist.lower(), j, 
 genius_verify_art_song_found_reformat[i][j]["obj"].title.lower()) 
 for i in genius_verify_art_song_found_reformat.keys() for j in genius_verify_art_song_found_reformat[i].keys()],key = lambda x: x[0])[:100]

[(0.5070422535211268,
  'the genies',
  'the wiggles',
  "Who's That Knocking",
  'dorothy the dinosaur (tell me who is that knocking?)'),
 (0.5102040816326531,
  'melissa manchester',
  'melissa manchester',
  'Theme From Ice Castles (Through The Eyes Of Love)',
  'through the eyes of love (theme from ice castles)'),
 (0.5106382978723404,
  'albert hammond',
  'albert hammond',
  'Air Disaster',
  "i don't wana die in an air disaster"),
 (0.5142857142857142,
  'shirley ellis',
  'shirley ellis',
  'Soul Time',
  'soul time - single version'),
 (0.5142857142857142,
  'the isley brothers',
  'the isley brothers',
  'That Lady (Part 1)',
  'that lady (part 1 & 2) - ahmir "?uest love" thompson'),
 (0.5151515151515151,
  'sammy kershaw & lorrie morgan',
  'sammy kershaw',
  'Maybe Not Tonight',
  'maybe not tonight (sammy kershaw & lorrie morgan)'),
 (0.5161290322580645,
  'jay-z featuring babyface and foxy brown',
  'jay-z',
  'Sunshine',
  '(always be my) sunshine'),
 (0.5185185185185185

In [67]:
genius_verify_art_song_found_reformat[list(genius_verify_art_song_found_reformat.keys())[0]]["Few And Far Between"]["lyrics"]

"Times are far between, and few I bet, when we can look upon our lives without regret\nOf all the things I have done, you think I'm proud of everyone without exception?\n\n'Till you make your peace with yesterday, you'll never build a future\nI swear by what I say\nWhatever penance you do, decide what it's worth to you and then respect it\nHowever long it will take to weather your mistakes, why not accept it?\n\nMy hands, for now, are tied\nI'm a body frozen\nI'm a will that's paralyzed\nWhen will you ever set aside your pain and misery?\nNo matter how I beg, no matter how I wish or plead, you'll never be more than alive\nYou'll never do more than survive until you expect it\nDo you want to build a world with our lives?\nYou better soon decide or you can forget it\n\nMy hands, for now, are tied\nI'm a body frozen\nI'm a will that's paralyzed\n'Till you drop that heavy baggage you're dragging behind, there won't be room for us to both go this ride"

In [72]:
song = genius.search_song("Girls Like You", "Maroon 5")

In [73]:
song.artist

'Maroon 5'

In [38]:
song_title = "Hey, Little Girl"
artist_name = 'Foster Sylvers'

In [43]:
import re
import urllib.request
from bs4 import BeautifulSoup
 
def get_lyrics(artist,song_title):
    artist = artist.lower()
    song_title = song_title.lower()
    # remove all except alphanumeric characters from artist and song_title
    artist = re.sub('[^A-Za-z0-9]+', "", artist)
    song_title = re.sub('[^A-Za-z0-9]+', "", song_title)
    if artist.startswith("the"):    # remove starting 'the' from artist e.g. the who -> who
        artist = artist[3:]
    url = "http://azlyrics.com/lyrics/"+artist+"/"+song_title+".html"
    
    try:
        content = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(content, 'html.parser')
        lyrics = str(soup)
        # lyrics lies between up_partition and down_partition
        up_partition = '<!-- Usage of azlyrics.com content by any third-party lyrics provider is prohibited by our licensing agreement. Sorry about that. -->'
        down_partition = '<!-- MxM banner -->'
        lyrics = lyrics.split(up_partition)[1]
        lyrics = lyrics.split(down_partition)[0]
        lyrics = lyrics.replace('<br>','').replace('</br>','').replace('</div>','').strip()
        return lyrics
    except Exception as e:
        return "Exception occurred \n" +str(e)

In [46]:
get_lyrics("Maroon 5", "Girls Like you").split("<br/>\n")

['Spent 24 hours, I need more hours with you',
 'You spent the weekend getting even, ooh',
 'We spent the late nights making things right between us',
 "But now it's all good, babe",
 'Roll that Backwood, babe',
 'And play me close',
 '',
 "'Cause girls like you run 'round with guys like me",
 "'Til sundown when I come through",
 'I need a girl like you, yeah yeah',
 'Girls like you love fun, and yeah, me too',
 'What I want when I come through',
 'I need a girl like you, yeah yeah',
 '',
 'Yeah yeah yeah, yeah yeah yeah',
 'I need a girl like you, yeah yeah',
 'Yeah yeah yeah, yeah yeah yeah',
 'I need a girl like you',
 '',
 'I spent last night on the last flight to you',
 'Took a whole day up trying to get way up, ooh',
 'We spent the daylight trying to make things right between us',
 "But now it's all good, babe",
 'Roll that Backwood, babe',
 'And play me close',
 '',
 "'Cause girls like you run 'round with guys like me",
 "'Til sundown when I come through",
 'I need a girl like y

In [70]:
ratio("My Kid".lower(), "I lost My Kid".lower())

0.631578947368421

In [77]:
BB.head()

,url,WeekID,Week Position,Song,Performer,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart
0,http://www.billboard.com/charts/hot-100/2012-1...,12/15/2012,21,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,28.0,21,11
1,http://www.billboard.com/charts/hot-100/2012-1...,12/22/2012,14,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,21.0,14,12
2,http://www.billboard.com/charts/hot-100/2012-1...,12/29/2012,12,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,14.0,12,13
3,http://www.billboard.com/charts/hot-100/2013-0...,1/5/2013,11,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,12.0,11,14
4,http://www.billboard.com/charts/hot-100/2013-0...,1/12/2013,15,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,11.0,11,15
